In [1]:
from simulation.notebook_utils import *
from simulation.utils import *

import busybee

In [2]:
# [!] CHANGE `24` to `1` for shorter simulations
SIM_TIME_MS = 24 * 3_600 * 1_000

#
# Global config for both ONLINE and OFFLINE
#

DEFAULT_CONFIG = NotebookSimulationConfig(
    loopix_kwargs= {
        'user_rate_pull': 1,
        'user_rate_payload': 2,
        'user_rate_drop': 2,
        'user_rate_loop': 2,
        'user_rate_delay': 3,
        'mix_rate_loop': 2,
        'mix_rate_loop_delay':3,
    },
    app_kwargs= {
        'init_rate_per_second': 1/30,
        'multi_message': 1,
        'heavy_user_percentage': 20,
        'heavy_user_weight': 4,
    }
)

SLOW_RATE = 1/30

CONFIG_NAMES_TO_CONFIGS = {
#     'r15': DEFAULT_CONFIG,
    'rs': DEFAULT_CONFIG.derive_new(
        delta_app_kwargs = {
            'init_rate_per_second': SLOW_RATE,
            'multi_message': 1,
        }),
    'rs-m2': DEFAULT_CONFIG.derive_new(
        delta_app_kwargs = {
            'init_rate_per_second': SLOW_RATE,
            'multi_message': 2,
        }),
    'rs-m4': DEFAULT_CONFIG.derive_new(
        delta_app_kwargs = {
            'init_rate_per_second': SLOW_RATE,
            'multi_message': 4,
        }),
    'rs-m8': DEFAULT_CONFIG.derive_new(
        delta_app_kwargs = {
            'init_rate_per_second': SLOW_RATE,
            'multi_message': 8,
        }),
    'rs-m16': DEFAULT_CONFIG.derive_new(
        delta_app_kwargs = {
            'init_rate_per_second': SLOW_RATE,
            'multi_message': 16,
        }),
    'rs-m32': DEFAULT_CONFIG.derive_new(
        delta_app_kwargs = {
            'init_rate_per_second': SLOW_RATE,
            'multi_message': 32,
        }),
    'rs-l1': DEFAULT_CONFIG.derive_new(
        delta_loopix_kwargs = {
            'user_rate_payload': 2,
            'user_rate_drop': 2,
            'user_rate_loop': 2,
            'user_rate_delay': 3,
            'mix_rate_loop': 2,
            'mix_rate_loop_delay':3,
        },
        delta_app_kwargs = {
            'init_rate_per_second': SLOW_RATE,
        }),
    'rs-l2': DEFAULT_CONFIG.derive_new(
        delta_loopix_kwargs = {
            'user_rate_payload': 2/2,
            'user_rate_drop': 2/2,
            'user_rate_loop': 2/2,
            'user_rate_delay': 3/2,
            'mix_rate_loop': 2/2,
            'mix_rate_loop_delay':3/2,
        },
        delta_app_kwargs = {
            'init_rate_per_second': SLOW_RATE,
        }),
    'rs-l4': DEFAULT_CONFIG.derive_new(
        delta_loopix_kwargs = {
            'user_rate_payload': 2/4,
            'user_rate_drop': 2/4,
            'user_rate_loop': 2/4,
            'user_rate_delay': 3/4,
            'mix_rate_loop': 2/4,
            'mix_rate_loop_delay':3/4,
        },
        delta_app_kwargs = {
            'init_rate_per_second': SLOW_RATE,
        }),
    'rs-l8': DEFAULT_CONFIG.derive_new(
        delta_loopix_kwargs = {
            'user_rate_payload': 2/8,
            'user_rate_drop': 2/8,
            'user_rate_loop': 2/8,
            'user_rate_delay': 3/8,
            'mix_rate_loop': 2/8,
            'mix_rate_loop_delay':3/8,
        },
        delta_app_kwargs = {
            'init_rate_per_second': SLOW_RATE,
        }), 
    'rs-l16': DEFAULT_CONFIG.derive_new(
        delta_loopix_kwargs = {
            'user_rate_payload': 2/16,
            'user_rate_drop': 2/16,
            'user_rate_loop': 2/16,
            'user_rate_delay': 3/16,
            'mix_rate_loop': 2/16,
            'mix_rate_loop_delay':3/16,
        },
        delta_app_kwargs = {
            'init_rate_per_second': SLOW_RATE,
        }),  
}

In [3]:
DEFAULT_SCHEDULES = ['gen_os10', 'gen_os15', 'gen_os20']

all_schedules = {k: read_compressed_bool_schedules("input/schedules_%s.txt.gz" % k) for k in DEFAULT_SCHEDULES}
all_schedules['online'] = None
print("Loaded schedules")

Loaded schedules


In [4]:
def pickle_to_file(args):
    pickle_down({args[0]: args[1]})

def create_sims(groupsizes, configs, schedules, strategies):
    sims = dict()
    for m, group_sizes in groupsizes:

        for config_name in configs:
            config = CONFIG_NAMES_TO_CONFIGS[config_name]

            for schedule_name in schedules:
                schedule = all_schedules[schedule_name]

                for strategy_name in strategies:
                        sim = create_scenario(m, group_sizes, config, strategy_name, SIM_TIME_MS, schedule if schedule else [])
                        name = get_name_for_sim(sim, config_name, schedule_name, strategy_name)

                        sims[name] = sim
                        sim.name = name.replace('.input', '').upper()


    _ = busybee.map(pickle_to_file, [(k, v) for k, v in sims.items()], processes=4)
    print("Saved %d files" % len(sims))

In [5]:
#
# ONLINE scenarios
#

ONLINE_GROUP_SIZES = [
    (32, [32]),
    (45, [45]),
    (64, [64]),
    (91, [91]),
    (128, [128]),
    (181, [181]),
    (256, [256]),
]
ONLINE_CONFIGS = ['rs']
ONLINE_SCHEDULES = ['online']
ONLINE_STRATEGIES = ['unicast', 'rollercoaster-k1-p1-notimeout', 'rollercoaster-k2-p2-notimeout']

create_sims(ONLINE_GROUP_SIZES, ONLINE_CONFIGS, ONLINE_SCHEDULES, ONLINE_STRATEGIES)

BusyBee: Start processing 21 items with 4 processes...
BusyBee: 11/21, 52.4% (avg: 67ms cpu, rem: 0.2s)
BusyBee: Finished processing 21 items in 0.9s (avg: 73ms cpu)
Saved 21 files


In [6]:
#
# OFFLINE scenarios
#

OFFLINE_GROUP_SIZES = [
    (128, [128]),
    (256, [256]),
]
OFFLINE_SCHEDULES = ['gen_os10', 'gen_os15', 'gen_os20']
OFFLINE_CONFIGS = ['rs']
OFFLINE_SCHEDULES = DEFAULT_SCHEDULES[:]
OFFLINE_STRATEGIES = [
    'unicast',
    'rollercoaster-k2-p2-notimeout',
    'rollercoaster-k1-p1-timeout10x-dropoffline',
    'rollercoaster-k2-p2-timeout10x-dropoffline',
]

# and for the histograms
OFFLINE_HIST_GROUP_SIZES = [(128, [128])]
OFFLINE_HIST_STRATEGIES = OFFLINE_STRATEGIES + ['rollercoaster-k2-p2-timeout10x'] 

create_sims(OFFLINE_GROUP_SIZES, OFFLINE_CONFIGS, OFFLINE_SCHEDULES, OFFLINE_STRATEGIES)
create_sims(OFFLINE_HIST_GROUP_SIZES, OFFLINE_CONFIGS, OFFLINE_SCHEDULES, OFFLINE_HIST_STRATEGIES)

BusyBee: Start processing 24 items with 4 processes...
BusyBee: 12/24, 50.0% (avg: 362ms cpu, rem: 1.1s)
BusyBee: 18/24, 75.0% (avg: 372ms cpu, rem: 0.6s)
BusyBee: Finished processing 24 items in 6.6s (avg: 363ms cpu)
Saved 24 files
BusyBee: Start processing 15 items with 4 processes...
BusyBee:  8/15, 53.3% (avg: 657ms cpu, rem: 1.1s)
BusyBee: Finished processing 15 items in 4.8s (avg: 607ms cpu)
Saved 15 files


In [7]:
#
# MULTIGROUP scenarios
#

MULTIGROUP_SIZE = 128
MULTIGROUP_GROUP_SIZES = [
    (MULTIGROUP_SIZE, [MULTIGROUP_SIZE]*1),
    (MULTIGROUP_SIZE, [MULTIGROUP_SIZE]*2),
    (MULTIGROUP_SIZE, [MULTIGROUP_SIZE]*4),
    (MULTIGROUP_SIZE, [MULTIGROUP_SIZE]*8),
    (MULTIGROUP_SIZE, [MULTIGROUP_SIZE]*16),
]
MULTIGROUP_CONFIGS = ['rs']
MULTIGROUP_SCHEDULES = ['online', 'gen_os15']
MULTIGROUP_STRATEGIES = [
    'unicast',
    'rollercoaster-k1-p1-timeout10x-dropoffline', 
    'rollercoaster-k2-p2-timeout10x-dropoffline', 
]

create_sims(MULTIGROUP_GROUP_SIZES, MULTIGROUP_CONFIGS, MULTIGROUP_SCHEDULES, MULTIGROUP_STRATEGIES)

BusyBee: Start processing 30 items with 4 processes...
BusyBee: 11/30, 36.7% (avg: 519ms cpu, rem: 2.5s)
BusyBee: 15/30, 50.0% (avg: 455ms cpu, rem: 1.7s)
BusyBee: Finished processing 30 items in 9.1s (avg: 370ms cpu)
Saved 30 files


In [8]:
#
# MULTIMESSAGE scenarios
#

MULTIMESSAGE_GROUP_SIZES = [
    (128, [128])
]
MULTIMESSAGE_CONFIGS = ['rs', 'rs-m2', 'rs-m4', 'rs-m8', 'rs-m16', 'rs-m32']
MULTIMESSAGE_SCHEDULES = ['online', 'gen_os15']
MULTIMESSAGE_STRATEGIES = [
    'unicast',
    'rollercoaster-k1-p1-timeout10x-dropoffline', 
    'rollercoaster-k2-p2-timeout10x-dropoffline', 
]

create_sims(MULTIMESSAGE_GROUP_SIZES, MULTIMESSAGE_CONFIGS, MULTIMESSAGE_SCHEDULES, MULTIMESSAGE_STRATEGIES)

BusyBee: Start processing 36 items with 4 processes...
BusyBee: 18/36, 50.0% (avg: 258ms cpu, rem: 1.2s)
BusyBee: 24/36, 66.7% (avg: 240ms cpu, rem: 0.7s)
BusyBee: Finished processing 36 items in 7.0s (avg: 215ms cpu)
Saved 36 files


In [9]:
#
# P scenarios
#

P_GROUP_SIZES = [
    (128, [128]),
]
P_CONFIGS = ['rs', 'rs-l2', 'rs-l4', 'rs-l8', 'rs-l16']
P_SCHEDULES = ['online', 'gen_os10', 'gen_os15', 'gen_os20']
P_STRATEGIES = [
    'rollercoaster-k1-p1-timeout10x-dropoffline', 
    'rollercoaster-k2-p2-timeout10x-dropoffline', 
    'rollercoaster-k4-p4-timeout10x-dropoffline', 
    'rollercoaster-k8-p8-timeout10x-dropoffline', 
    'rollercoaster-k16-p16-timeout10x-dropoffline', 
    'rollercoaster-k2-p1-timeout10x-dropoffline', 
    'rollercoaster-k4-p1-timeout10x-dropoffline', 
    'rollercoaster-k8-p1-timeout10x-dropoffline', 
    'rollercoaster-k16-p1-timeout10x-dropoffline', 
]

create_sims(P_GROUP_SIZES, P_CONFIGS, P_SCHEDULES, P_STRATEGIES)

BusyBee: Start processing 180 items with 4 processes...
BusyBee:  20/180, 11.1% (avg: 302ms cpu, rem: 12.1s)
BusyBee:  32/180, 17.8% (avg: 354ms cpu, rem: 13.1s)
BusyBee:  55/180, 30.6% (avg: 306ms cpu, rem: 9.6s)
BusyBee:  63/180, 35.0% (avg: 330ms cpu, rem: 9.7s)
BusyBee:  90/180, 50.0% (avg: 315ms cpu, rem: 7.1s)
BusyBee:  91/180, 50.6% (avg: 314ms cpu, rem: 7.0s)
BusyBee: 107/180, 59.4% (avg: 319ms cpu, rem: 5.8s)
BusyBee: 130/180, 72.2% (avg: 300ms cpu, rem: 3.8s)
BusyBee: 144/180, 80.0% (avg: 307ms cpu, rem: 2.8s)
BusyBee: 164/180, 91.1% (avg: 302ms cpu, rem: 1.2s)
BusyBee: 176/180, 97.8% (avg: 310ms cpu, rem: 0.3s)
BusyBee: Finished processing 180 items in 54.7s (avg: 311ms cpu)
Saved 180 files
